In [ ]:
%load_ext autoreload
%autoreload 2
from shapley.shapley_fda import ShapleyFda
from images import images_path
from matplotlib.ticker import FormatStrFormatter
from utils.utils_workflow import plot_shapley_function, plot_shapley_value
import matplotlib.pyplot as plt
import numpy as np
import os
import seaborn as sns

In [ ]:
def generate_data(m, T, set_abscissas, mu, mu_range=None):
    X = np.full(shape=(m, T), fill_value=np.nan)
    if mu_range:
        mu_vector = np.random.uniform(low=mu_range[0], high=mu_range[1],size=m)
    else:
        mu_vector = [mu for _ in range(m)]
    for i in range(m):
        x_i = np.ravel(np.array([(x - mu_vector[i]) **2 for x in set_abscissas]))
        X[i] = x_i
    return X

def predict_fn(X):
    min_x = np.min(X, axis=1) 
    return min_x

In [ ]:
save_figures = True
T = 100
m = 100
interval = [0, 1]
#mu_range = [0.45, 0.55]
mu_range = None
mu = 0.5
step = (interval[1] - interval[0])/(T - 1)
set_abscissas = np.arange(interval[0], interval[1] + step, step)
random_walk = np.random.normal(loc=0, scale=0.01, size=(m, T))
deterministic = generate_data(m=m, T=T, set_abscissas=set_abscissas, mu_range=mu_range, mu=mu)
functional_data = deterministic + random_walk
target = predict_fn(functional_data)


In [ ]:
aspect_ratio = 6.4/4.8
fig_size = [8*6.4/4.8, 8]
fig, ax = plt.subplots()
ax.tick_params(axis='x', labelsize=24)
ax.tick_params(axis='y', labelsize=24)
ax.set_xlabel(r"$t$", fontsize=28)
ax.set_ylabel(r"$\mathcal{X}_j(t)$", fontsize=28)
fig.set_size_inches(fig_size)
fig.set_facecolor("none")
ax.set_facecolor("white")
zzz = plt.plot(set_abscissas, functional_data.T)
if save_figures:
    fig.savefig(os.path.join(images_path, "toy_functional_covariates.pdf"), format="pdf")
    plt.close()

In [ ]:
argmin_x = set_abscissas[np.argmin(functional_data, axis=1)]
fig, ax = plt.subplots()
ax.tick_params(axis='x', labelsize=24)
ax.tick_params(axis='y', labelsize=24)
ax.set_xlabel(r"$t$", fontsize=28)
ax.set_ylabel("Density", fontsize=28)
fig.set_size_inches(fig_size)
fig.set_facecolor("none")
ax.set_facecolor("white")
sns.kdeplot(argmin_x)
ax.set_xlim(0, 1)
if save_figures:
    fig.savefig(os.path.join(images_path, "toy_density_argmin.pdf"), format="pdf")
    plt.close()

In [ ]:
num_permutations = 120
num_intervals = 5
shapley_fda = ShapleyFda(
    X=functional_data,
    abscissa_points=set_abscissas,
    target=target,
    domain_range=interval,
    verbose=False,
)

shapley_values = shapley_fda.compute_shapley_value(
    num_permutations=num_permutations,
    predict_fns=predict_fn,
    num_intervals=num_intervals,
    compute_mrmr_r2=False,
    compute_mrmr_distance_correlation=False,
)

fig, ax = plt.subplots(1)
plot_shapley_value(
    obj=shapley_values,
    domain_range=interval,
    display_legend=False,
    translation_dict={0: None},
)
ax.tick_params(axis='x', labelsize=28)
ax.tick_params(axis='y', labelsize=28)
fig.set_size_inches(fig_size)
ax.set_facecolor("white")
fig.set_facecolor("none")
ax.set_xlabel(r"$t$", fontsize=30)
ax.set_ylabel(r"$\mathcal{R}(t)$", fontsize=30)
ax.yaxis.set_major_formatter(FormatStrFormatter('%.1f'))
if save_figures:
    fig.savefig(os.path.join(images_path, "toy_polygonal.pdf"), format="pdf")
    plt.close()

In [ ]:
fig, ax = plt.subplots(1)
plot_shapley_function(
    obj=shapley_values,
    domain_range=interval,
    plot_h_line=True,
    plot_v_line=True,
    display_legend=False,
    translation_dict={0: None},
)
ax.tick_params(axis='x', labelsize=28)
ax.tick_params(axis='y', labelsize=28)
fig.set_size_inches(fig_size)
ax.set_facecolor("white")
fig.set_facecolor("none")
ax.set_xlabel(r"$t$", fontsize=30)
ax.set_ylabel(r"$\mathcal{R}(t)$", fontsize=30)
ax.yaxis.set_major_formatter(FormatStrFormatter('%.1f'))
if save_figures:
    fig.savefig(os.path.join(images_path, "toy_histogram.pdf"), format="pdf")
    plt.close()